In [45]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision import transforms,datasets
from torch.utils.data import DataLoader


In [46]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [47]:

transform = transforms.Compose([
    transforms.ToTensor(),  # Converts PIL Image to Tensor and scales [0, 255] to [0.0, 1.0]
    # No need to add channel dimension as ToTensor() handles it for standard datasets
])

# Load your dataset (e.g., MNIST)
train_dataset = datasets.MNIST(root='data', train=True, download=True, transform=transform)

# Create a DataLoader
batch_size = 100  # Adjust as needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

# Fetch the first batch
all_images, all_labels = next(iter(train_loader))  # all_images shape: [100, 1, 28, 28]
print(all_images.shape)

torch.Size([100, 1, 28, 28])


In [52]:
class VAEEncoder(nn.Module):
    def __init__(self,latent_dimension):
        super(VAEEncoder,self).__init__()
        self.conv1 = nn.Conv2d(1,32,kernel_size=3,stride=1,padding=2) 
        self.conv2 = nn.Conv2d(32,64,kernel_size=3,stride=1,padding=2)
        self.pool = nn.MaxPool2d(kernel_size=2) 

        self.fc1 = nn.Linear(64*8*8,120)
        self.fc2 = nn.Linear(120,100)
        self.fc_mean = nn.Linear(100,latent_dimension)
        self.fc_logvar = nn.Linear(100,latent_dimension)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
    def forward(self,x):
        x = self.pool(self.conv1(x))
        x = self.pool(self.conv2(x))
        x  = x.view(x.size(0),-1) # flatten my image
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))

        mean = self.fc_mean(x)
        log_var = self.fc_logvar(x)
        x = x.view(-1,1)
        return mean,log_var,x
    def reparamtize_trick(self,mean,log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        z = mean+eps*std
        return z   
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Instantiate the encoder with correct input channels (1 for MNIST)
encoder = VAEEncoder(100).to(device)

# Move data to device
all_images = all_images.to(device)  # all_images shape: [100, 1, 28, 28]

# Forward pass through the encoder
mu, logvar,x = encoder.forward(all_images)
z = encoder.reparamtize_trick(mu,logvar)
print(z.shape)

torch.Size([100, 100])


In [53]:
class approximated_probability(VAEEncoder):
    def __init__(self):
        print('We are finding the approximated probability')
    def probability_desity(self,mean,log_var):
        z = super().reparamtize_trick(mean,log_var)
        
        var = torch.exp(log_var)
        cov = torch.diag_embed(var)
        cov_det = torch.linalg.det(cov)
        cov_inv = torch.linalg.inv(cov)
        norm = 1/torch.sqrt(torch.tensor(2*torch.pi))*cov_det
        diff = z-mean
        exponent = torch.exp(-0.5*diff@cov_inv*diff)
        return norm*exponent
obj = approximated_probability()
aprox_prob = obj.probability_desity(mu,logvar)

We are finding the approximated probability
torch.Size([100, 100, 100])
